In [1]:
import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

In [2]:
input_ = pd.read_excel('./menu.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
input_['Url'].tolist()

总数量：355


['https://rotomaster.com/products/performance-upgrades/performance-upgrade-turbo-1998-1999-ford-7-3l-billet-wheel-w-o-pedestal-a1380153n/',
 'https://rotomaster.com/products/performance-upgrades/performance-upgrade-turbo-1999-2003-7-3l-w-billet-wheel-ford-a1380152n/',
 'https://rotomaster.com/products/performance-upgrades/performance-upgrade-turbo-1999-2003-ford-7-3l-gtp38-w-o-pedestal-a1380107n/',
 'https://rotomaster.com/products/performance-upgrades/performance-upgrade-turbo-1999-2003-ford-stage-2-w-66mm-88-mm-billet-wheel-7-3l-a1380155n/',
 'https://rotomaster.com/products/performance-upgrades/performance-upgrade-turbo-1999-2003-ford-stage-2-w-66mm-88-mm-compressor-wheel-7-3l-a1380105n/',
 'https://rotomaster.com/turbochargers/reman-turbo-for-1980-1981-38l-chevrolet-a8300102r/',
 'https://rotomaster.com/turbochargers/reman-turbo-for-1980-1983-38l-buick-a8030113r/',
 'https://rotomaster.com/turbochargers/reman-turbo-for-1982-1984-16l-renault-a8320101r/',
 'https://rotomaster.com/tur

In [3]:
crawler_status = 'error'

crawler_status

'error'

In [4]:
import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

In [5]:
a = 0

resp = requests.get(input_.loc[a, 'Url'],
                    headers=UA.get_User_Agent_Requests(),
                    proxies=Proxy.get_Proxy_Requests()).text

resp

'<!DOCTYPE html>\r\n<html class="no-js" lang="en">\r\n    <head>\r\n        <script>\r\n        </script>\r\n\r\n        <title>Performance Upgrade Turbo 1998-1999 Ford 7.3L Billet Wheel, w/o Pedestal A1380153N</title>\r\n        <link rel="dns-prefetch preconnect" href="https://cdn11.bigcommerce.com/s-ouot3zjaxm" crossorigin><link rel="dns-prefetch preconnect" href="https://fonts.googleapis.com/" crossorigin><link rel="dns-prefetch preconnect" href="https://fonts.gstatic.com/" crossorigin>\r\n        <meta property="og:url" content="https://rotomaster.com/products/performance-upgrades/performance-upgrade-turbo-1998-1999-ford-7-3l-billet-wheel-w-o-pedestal-a1380153n/" /><meta property="og:site_name" content="Rotomaster" /><meta name="description" content="Rotomaster Performance Upgrade Turbo 1998-1999 Ford 7.3L Billet Wheel, w/o Pedestal A1380153N"><link rel=\'canonical\' href=\'https://rotomaster.com/products/performance-upgrades/performance-upgrade-turbo-1998-1999-ford-7-3l-billet-wh

In [6]:
from bs4 import BeautifulSoup
from lxml import etree

In [7]:
soup = BeautifulSoup(resp, 'lxml')
html = etree.HTML(str(soup))

with open('./part.html', 'w', encoding='UTF-8') as file:
    file.write(str(soup))

print(soup.prettify())

<!DOCTYPE html>
<html class="no-js" lang="en">
 <head>
  <script>
  </script>
  <title>
   Performance Upgrade Turbo 1998-1999 Ford 7.3L Billet Wheel, w/o Pedestal A1380153N
  </title>
  <link crossorigin="" href="https://cdn11.bigcommerce.com/s-ouot3zjaxm" rel="dns-prefetch preconnect"/>
  <link crossorigin="" href="https://fonts.googleapis.com/" rel="dns-prefetch preconnect"/>
  <link crossorigin="" href="https://fonts.gstatic.com/" rel="dns-prefetch preconnect"/>
  <meta content="https://rotomaster.com/products/performance-upgrades/performance-upgrade-turbo-1998-1999-ford-7-3l-billet-wheel-w-o-pedestal-a1380153n/" property="og:url"/>
  <meta content="Rotomaster" property="og:site_name"/>
  <meta content="Rotomaster Performance Upgrade Turbo 1998-1999 Ford 7.3L Billet Wheel, w/o Pedestal A1380153N" name="description"/>
  <link href="https://rotomaster.com/products/performance-upgrades/performance-upgrade-turbo-1998-1999-ford-7-3l-billet-wheel-w-o-pedestal-a1380153n/" rel="canonical"/

In [8]:
title = html.xpath('//h1[@itemprop="name"]/text()')[0].strip()

title

'Performance Upgrade Turbo 1998-1999 Ford 7.3L Billet Wheel, w/o Pedestal A1380153N'

In [9]:
src = html.xpath('//div[@class="productView-img-container"]/a/@href')[0].strip()

src

'https://cdn11.bigcommerce.com/s-ouot3zjaxm/images/stencil/1280x1280/products/438/1784/A1380153N-fro__09375.1657294890.jpg?c=1'

In [10]:
list_dt =html.xpath('//dl[@class="productView-info"]/dt/text()')
list_dd = html.xpath('//dl[@class="productView-info"]/dd/text()')

list_dt, list_dd

(['SKU:', 'Part Type:', 'Product Condition:', 'Application Type:'],
 ['A1380153N', 'Turbocharger', 'New', 'Performance Upgrade'])

In [11]:
list_tr = html.xpath('//div[@id="tabDescription-content"]/table/tbody/tr')

list_tr

[<Element tr at 0x120f78980>,
 <Element tr at 0x120fd4f40>,
 <Element tr at 0x106dbaf40>]

In [12]:
df_temp = pd.DataFrame([{'序号': int(input_.loc[a, 'No.']),
                         'Title': title,
                         'Pic': '',
                         'Url': input_.loc[a, 'Url'],
                         'Src': src}])

for dt, dd in zip(list_dt, list_dd):
    df_temp.loc[0, dt.strip()[:-1].lower()] = dd.strip()
    
for tr in list_tr:
    if tr.xpath('./td[1]/text()')[0].strip() == 'Application':
        df_temp.loc[0, tr.xpath('./td[1]/text()')[0].strip()] = '\n'.join([vehicle.strip() for vehicle in tr.xpath('./td[2]/p/text()')])
    else:
        df_temp.loc[0, tr.xpath('./td[1]/text()')[0].strip()] = ';'.join([td.strip() for td in tr.xpath('./td[2]/text()')])
    
output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

len(output_correct)

1

In [13]:
crawler_status = 'ok'

crawler_status

'ok'

In [14]:
if crawler_status == 'error':
    df_temp = pd.DataFrame([{'No.': int(input_.loc[a, 'No.']),
                             'Url': input_.loc[a, 'Url']}])
    
    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

len(output_error)

0

In [15]:
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['序号'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./test_part.xlsx', index=False)

output_correct

,序号,Title,Pic,Url,Src,sku,part type,product condition,application type,Package Contents,Application,OE Part Number
0,1,Performance Upgrade Turbo 1998-1999 Ford 7.3L ...,,https://rotomaster.com/products/performance-up...,https://cdn11.bigcommerce.com/s-ouot3zjaxm/ima...,A1380153N,Turbocharger,New,Performance Upgrade,"Turbocharger, Seal(s), Syringe",1999 Ford E Super Duty 7.3L w/Actuator Before ...,5C3Z6V682BRM


In [16]:
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['No.'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./test_part_error.xlsx', index=False)

output_error

""
